### DEinstagram crawling to csv
   
- daily_gamsung.csv 
- gamsung.bnb.csv 
- gamsung_curation.csv 
- hi.stay.tour.csv 
- rest_behappyhere.csv 
- sookso.diary.csv 
- sookso.hada.csv

1. 숙소이름 추출  
2. 계정별 숙소 리스트 취합  
3. 통합 숙소리스트를 기준으로 content 병합  
4. like 수 합치기  
5. content 일괄 전처리(토큰화, 불용어제거)  

### 최종 데이터 셋  
columns = 숙소명, content(통합), like(통합), place(api), tags(통합), 게시수(중복값)


In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import re

import time
import os
import sys
import urllib.request
import json

from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import array_contains, udf
from datetime import datetime

from datetime import datetime, timedelta
from pymongo import MongoClient
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
from collections import Counter
from datetime import datetime



spark = SparkSession\
        .builder\
        .appName('Python Spark SQL basic example')\
        .getOrCreate()

sc = spark.sparkContext

In [4]:
nowtime = '2021-10-05'

daily_gamsung_DFP = spark.read.parquet(f"hdfs://localhost:9000/data/insta/daily_gamsung_{nowtime}.parquet")
daily_gamsung_DFP = daily_gamsung_DFP.drop('Unnamed: 0','unix')
daily_gamsung = daily_gamsung_DFP.toPandas()

gamsung_bnb_DFP = spark.read.parquet(f"hdfs://localhost:9000/data/insta/gamsung_bnb_{nowtime}.parquet")
gamsung_bnb_DFP = gamsung_bnb_DFP.drop('Unnamed: 0','unix')
gamsung_bnb = gamsung_bnb_DFP.toPandas()

gamsung_curation_DFP = spark.read.parquet(f"hdfs://localhost:9000/data/insta/gamsung_curation_{nowtime}.parquet")
gamsung_curation_DFP = gamsung_curation_DFP.drop('Unnamed: 0','unix')
gamsung_curation = gamsung_curation_DFP.toPandas()

hi_stay_tour_DFP = spark.read.parquet(f"hdfs://localhost:9000/data/insta/hi_stay_tour_{nowtime}.parquet")
hi_stay_tour_DFP = hi_stay_tour_DFP.drop('Unnamed: 0','unix')
hi_stay_tour = hi_stay_tour_DFP.toPandas()

rest_behappyhere_DFP = spark.read.parquet(f"hdfs://localhost:9000/data/insta/rest_behappyhere_{nowtime}.parquet")
rest_behappyhere_DFP = rest_behappyhere_DFP.drop('Unnamed: 0','unix')
rest_behappyhere = rest_behappyhere_DFP.toPandas()

sookso_diary_DFP = spark.read.parquet(f"hdfs://localhost:9000/data/insta/sookso_diary_{nowtime}.parquet")
sookso_diary_DFP = sookso_diary_DFP.drop('Unnamed: 0','unix')
sookso_diary = sookso_diary_DFP.toPandas()

sookso_hada_DFP = spark.read.parquet(f"hdfs://localhost:9000/data/insta/sookso_hada_{nowtime}.parquet")
sookso_hada_DFP = sookso_hada_DFP.drop('Unnamed: 0','unix')
sookso_hada = sookso_hada_DFP.toPandas()

### daily_gamsung(완료)

In [5]:
def daily_gamsung_extract(x):
    p = re.compile('\"[가-힣\t\n\r\f\v\s\_가-힣]+\"')
    m = p.findall(x)
    try:
        return m[0][1:-1]
    except:
        return np.nan

def convert(x):
    if x:
        return int(x.replace(',',''))
    else:
        return 0    
    
daily_gamsung_name = daily_gamsung.content.apply(lambda x : daily_gamsung_extract(x))
daily_gamsung['name'] = daily_gamsung_name
daily_gamsung = daily_gamsung[~(daily_gamsung['name'].isna())]

# name data가 없는 index 지우기
indexNames = daily_gamsung[daily_gamsung['name'] == ''].index
daily_gamsung.drop(indexNames, inplace=True) 

if daily_gamsung.like.dtype == 'O':
    daily_gamsung.like = daily_gamsung.like.apply(lambda x : convert(x))


# 1,2,3
dg_cnt = Counter(daily_gamsung.name)
li =[]
for name, cnt  in dg_cnt.items():
    if cnt > 1:
        li.append(name)   

# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = daily_gamsung.loc[daily_gamsung.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset

    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    imgUrl = ex.iloc[0,:].imgUrl
    name = ex.name[0]

    for i in range(len(ex)-1):
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags

    # 5.차례대로 기존 행 지워내기
    indexNames = daily_gamsung[daily_gamsung['name'] == ex.name[0]].index
    daily_gamsung.drop(indexNames, inplace=True) 
    daily_gamsung = daily_gamsung.append(pd.Series(data=[content,date,like,place,tags,imgUrl,name],index=daily_gamsung.columns),ignore_index=True)

dg_tmp = []
for i in daily_gamsung.name:
    dg_tmp.append(dg_cnt[i]-1)

daily_gamsung['overlap'] = dg_tmp

### gamsun.bnb

In [6]:
def gamsung_bnb_extract(x):
    p = re.compile("[\'|\"][가-힣]+[\'|\"]") 
    m = p.findall(x)
    try:
        return m[0][1:-1]
    except :
        return np.nan

gamsung_bnb_name = gamsung_bnb.content.apply(lambda x : gamsung_bnb_extract(x)) # gamsung_bnb_name = 
gamsung_bnb['name'] = gamsung_bnb_name
gamsung_bnb = gamsung_bnb[~(gamsung_bnb['name'].isna())]

# name data가 없는 index 지우기
indexNames = gamsung_bnb[gamsung_bnb['name'] == ''].index
gamsung_bnb.drop(indexNames, inplace=True) 

# like integet 변환 후 합치기

def convert(x):
    if x:
        return int(x.replace(',',''))
    else:
        return 0

if gamsung_bnb.like.dtype == 'O':
    gamsung_bnb.like = gamsung_bnb.like.apply(lambda x : convert(x))



# 1, 중복된 이름 검색 
# 2.개수가 1인거 이상 뽑아내기 
# 3.리스트만들기
gb_cnt = Counter(gamsung_bnb.name)      # Counter 모듈로 name data count
li =[]
for name, cnt  in gb_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
    if cnt > 1:
        li.append(name)

# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = gamsung_bnb.loc[gamsung_bnb.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset

    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    imgUrl = ex.iloc[0,:].imgUrl
    name = ex.name[0]

    for i in range(len(ex)-1):
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags


    # 5.차례대로 기존 행 지워내기
    indexNames = gamsung_bnb[gamsung_bnb['name'] == ex.name[0]].index
    gamsung_bnb.drop(indexNames, inplace=True) 
    gamsung_bnb = gamsung_bnb.append(pd.Series(data=[content,date,like,place,tags,imgUrl,name],index=gamsung_bnb.columns),ignore_index=True)

gb_tmp = []
for i in gamsung_bnb.name:
    gb_tmp.append(gb_cnt[i]-1)


gamsung_bnb['overlap'] = gb_tmp

### gamsung_curation(완료)

In [7]:

'''
'b'       boolean
'i'       (signed) integer
'u'       unsigned integer
'f'       floating-point
'c'       complex-floating point
'O'       (Python) objects
'S', 'a'  (byte-)string
'U'       Unicode
'V'       raw data (void)
'''
def gamsung_curation_extract(x):
    p = re.compile('\📍[가-힣]+') # 
    m = p.findall(x)
    try:
        return m[0][1:] 
    except :
        return ''

### gamsung_curation(완료)
gamsung_curation_name = gamsung_curation.content.apply(lambda x : gamsung_curation_extract(x))
gamsung_curation['name'] = gamsung_curation_name
gamsung_curation = gamsung_curation[~(gamsung_curation['name'].isna())]


# name data가 없는 index 지우기
indexNames = gamsung_curation[gamsung_curation['name'] == ''].index
gamsung_curation.drop(indexNames, inplace=True) 




if gamsung_curation.like.dtype == 'O':
    gamsung_curation.like = gamsung_curation.like.apply(lambda x : convert(x))


gc_cnt = Counter(gamsung_curation.name)      # Counter 모듈로 name data count
li =[]
for name, cnt  in gc_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
    if cnt > 1:
        li.append(name)        


# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = gamsung_curation.loc[gamsung_curation.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset

    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    imgUrl = ex.iloc[0,:].imgUrl
    name = ex.name[0]

    for i in range(len(ex)-1):
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags

    # 5.차례대로 기존 행 지워내기
    indexNames = gamsung_curation[gamsung_curation['name'] == ex.name[0]].index
    gamsung_curation.drop(indexNames, inplace=True) 
    gamsung_curation = gamsung_curation.append(pd.Series(data=[content,date,like,place,tags,imgUrl,name],index=gamsung_curation.columns),ignore_index=True)

gc_tmp = []
for i in gamsung_curation.name:
    gc_tmp.append(gc_cnt[i]-1)

gamsung_curation['overlap'] = gc_tmp


### hi.stay.tour.csv (완료)

In [8]:
def hi_stay_tour_extract(x):
    p = re.compile("\#[가-힣]+") # 
    m = p.findall(x)
    try:
        return m[0][1:] 
    except :
        return ''

### hi.stay.tour.csv (완료)
hi_stay_tour_name = hi_stay_tour.content.apply(lambda x : hi_stay_tour_extract(x))
hi_stay_tour['name'] = hi_stay_tour_name
hi_stay_tour = hi_stay_tour[~(hi_stay_tour['name'].isna())]


# name data가 없는 index 지우기
indexNames = hi_stay_tour[hi_stay_tour['name'] == ''].index
hi_stay_tour.drop(indexNames, inplace=True) 


if hi_stay_tour.like.dtype == 'O':
    hi_stay_tour.like = hi_stay_tour.like.apply(lambda x : convert(x))


# 1, 중복된 이름 검색 
# 2.개수가 1인거 이상 뽑아내기 
# 3.리스트만들기
hst_cnt = Counter(hi_stay_tour.name)      # Counter 모듈로 name data count
li =[]
for name, cnt  in hst_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
    if cnt > 1:
        li.append(name)  

# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = hi_stay_tour.loc[hi_stay_tour.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset

    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    imgUrl = ex.iloc[0,:].imgUrl
    name = ex.name[0]

    for i in range(len(ex)-1):
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags



    # 5.차례대로 기존 행 지워내기
    indexNames = hi_stay_tour[hi_stay_tour['name'] == ex.name[0]].index
    hi_stay_tour.drop(indexNames, inplace=True) 
    hi_stay_tour = hi_stay_tour.append(pd.Series(data=[content,date,like,place,tags,imgUrl,name],index=hi_stay_tour.columns),ignore_index=True)

hst_cnt_tmp = []
for i in hi_stay_tour.name:
    hst_cnt_tmp.append(hst_cnt[i]-1)

hi_stay_tour['overlap'] = hst_cnt_tmp

### rest_behappyhere.csv

In [9]:
def rest_behappyhere_extract(x):
    p = re.compile('\#[가-힣]+')
    m = p.findall(x)
    try:
        return m[1][1:]
    except:
        return np.nan

rest_behappyhere_name = rest_behappyhere.content.apply(lambda x : rest_behappyhere_extract(x))
rest_behappyhere['name'] = rest_behappyhere_name
rest_behappyhere = rest_behappyhere[~(rest_behappyhere['name'].isna())]

# name data가 없는 index 지우기
indexNames = rest_behappyhere[rest_behappyhere['name'] == ''].index
rest_behappyhere.drop(indexNames, inplace=True) 



if rest_behappyhere.like.dtype == 'O':
    rest_behappyhere.like = rest_behappyhere.like.apply(lambda x : convert(x))


# 1, 중복된 이름 검색 
# 2.개수가 1인거 이상 뽑아내기 
# 3.리스트만들기
rb_cnt = Counter(rest_behappyhere.name)      # Counter 모듈로 name data count
li =[]
for name, cnt  in rb_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
    if cnt > 1:
        li.append(name)


#####################################################
# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = rest_behappyhere.loc[rest_behappyhere.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset

    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    imgUrl = ex.iloc[0,:].imgUrl
    name = ex.name[0]

    for i in range(len(ex)-1):
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags



    # 5.차례대로 기존 행 지워내기
    indexNames = rest_behappyhere[rest_behappyhere['name'] == ex.name[0]].index
    rest_behappyhere.drop(indexNames, inplace=True) 
    rest_behappyhere = rest_behappyhere.append(pd.Series(data=[content,date,like,place,tags,imgUrl,name],index=rest_behappyhere.columns),ignore_index=True)

rb_cnt_tmp = []
for i in rest_behappyhere.name:
    rb_cnt_tmp.append(rb_cnt[i]-1)

rest_behappyhere['overlap'] = rb_cnt_tmp

### sookso.diary(완료)

In [10]:
def sookso_diary_extract(x):
    p = re.compile('\📍 [가-힣]+\s[가-힇]+|\📍[가-힣]+\s[가-힇]+')
    m = p.findall(x)
    try:
        return m[0].split(' ')[-1]
    except:
        return np.nan

sookso_diary_name = sookso_diary.content.apply(lambda x : sookso_diary_extract(x))
sookso_diary['name'] = sookso_diary_name
sookso_diary = sookso_diary[~(sookso_diary['name'].isna())]


# name data가 없는 index 지우기
indexNames = sookso_diary[sookso_diary['name'] == ''].index
sookso_diary.drop(indexNames, inplace=True) 



if sookso_diary.like.dtype == 'O':
    sookso_diary.like = sookso_diary.like.apply(lambda x : convert(x))


# 1, 중복된 이름 검색 
# 2.개수가 1인거 이상 뽑아내기 
# 3.리스트만들기
sd_cnt = Counter(sookso_diary.name)      # Counter 모듈로 name data count
li =[]
for name, cnt  in sd_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
    if cnt > 1:
        li.append(name)

# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = sookso_diary.loc[sookso_diary.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset

    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    imgUrl = ex.iloc[0,:].imgUrl
    name = ex.name[0]

    for i in range(len(ex)-1):
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags



    # 5.차례대로 기존 행 지워내기
    indexNames = sookso_diary[sookso_diary['name'] == ex.name[0]].index
    sookso_diary.drop(indexNames, inplace=True) 
    sookso_diary = sookso_diary.append(pd.Series(data=[content,date,like,place,tags,imgUrl,name],index=sookso_diary.columns),ignore_index=True)

sd_cnt_tmp = []
for i in sookso_diary.name:
    sd_cnt_tmp.append(sd_cnt[i]-1)

sookso_diary['overlap'] = sd_cnt_tmp

---

### sookso.hada (완료)

In [11]:
def sookso_hada_extract(x):
    p = re.compile('\#[가-힣]+')
    m = p.findall(x)
    if  m[1] == '#숙소하다': m[1], m[0] = m[0], m[1]
    return m[1][1:]


sookso_hada_name = sookso_hada.content.apply(lambda x : sookso_hada_extract(x))
sookso_hada['name'] = sookso_hada_name
sookso_hada = sookso_hada[~(sookso_hada['name'].isna())]

# name data가 없는 index 지우기
indexNames = sookso_hada[sookso_hada['name'] == ''].index
sookso_hada.drop(indexNames, inplace=True) 


# like integet 변환 후 합치기

if sookso_hada.like.dtype == 'O':
    sookso_hada.like = sookso_hada.like.apply(lambda x : convert(x))


# 1, 중복된 이름 검색 
# 2.개수가 1인거 이상 뽑아내기 
# 3.리스트만들기
sh_cnt = Counter(sookso_hada.name)      # Counter 모듈로 name data count
li =[]
for name, cnt  in sh_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
    if cnt > 1:
        li.append(name)        


# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = sookso_hada.loc[sookso_hada.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset

    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    imgUrl = ex.iloc[0,:].imgUrl
    name = ex.name[0]

    for i in range(len(ex)-1):
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags

    # 5.차례대로 기존 행 지워내기
    indexNames = sookso_hada[sookso_hada['name'] == ex.name[0]].index
    sookso_hada.drop(indexNames, inplace=True) 
    sookso_hada = sookso_hada.append(pd.Series(data=[content,date,like,place,tags,imgUrl,name],index=sookso_hada.columns),ignore_index=True)

sh_cnt_tmp = []
for i in sookso_hada.name:
    sh_cnt_tmp.append(sh_cnt[i]-1)

sookso_hada['overlap'] = sh_cnt_tmp


---

##  데이터 프레임 합치기

In [12]:
tot_dataset = daily_gamsung
for df in [gamsung_bnb, gamsung_curation, hi_stay_tour, rest_behappyhere, sookso_diary, sookso_hada]:
    tot_dataset = tot_dataset.append(df)

if tot_dataset.like.dtype == 'O':
    tot_dataset.like = tot_dataset.like.apply(lambda x : convert(x))

dg_cnt = Counter(tot_dataset.name)      # Counter 모듈로 name data count
li =[]
for name, cnt  in dg_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
    if cnt > 1:
        li.append(name)

# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = tot_dataset.loc[tot_dataset.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset

    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    imgUrl = ex.iloc[0,:].imgUrl
    name = ex.name[0]
    overlap = ex.iloc[0,:].overlap

    for i in range(len(ex)-1):
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags
        overlap += ex.iloc[i+1,:].overlap


    # 5.차례대로 기존 행 지워내기
    indexNames = tot_dataset[tot_dataset['name'] == ex.name[0]].index
    tot_dataset.drop(indexNames, inplace=True) 
    tot_dataset = tot_dataset.append(pd.Series(data=[content,date,like,place,tags,imgUrl,name,overlap],index=tot_dataset.columns),ignore_index=True)


In [13]:
tot_dataset

,content,date,like,place,tags,imgUrl,name,overlap
0,"포항_선바우집호미곶 오션뷰 한옥 독채민박""선바우집""'선바우집'은가족단위로 머물기 좋...",2021-09-30,34,포항 해안둘레길 선바우,"['#데일리감성포항', '#데일리감성_포항경상북도로', '#데일리감성_경북—————...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,선바우집,0
1,"밀양_석주재프라이빗한 한옥 숙소""석주재""'석주재'는밀양의 프라이빗한 한옥 감성숙소로...",2021-09-27,22,밀양,"['#데일리감성밀양', '#데일리감성_밀양경상남도로', '#데일리감성_경남—————...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,석주재,0
2,"여수_컴온인동남아 이국적 분위기의 감성 숙소""컴온인""'컴온인'은동남아의 이국적인 분...",2021-09-26,21,Yeosu,"['#데일리감성여수', '#데일리감성_여수', '#데일리감성_전남——————————...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,컴온인,0
3,"강화_문스톤펜션강화도에서 가장 감성적인 숙소""문스톤펜션""'문스톤펜션'은강화도 동막해...",2021-09-26,14,강화도,"['#데일리감성강화', '#데일리감성_강화경기도로', '#데일리감성_경기——————...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,문스톤펜션,0
4,"제주_고등어민박제주 동쪽 로맨틱한 감성 숙소""고등어민박""'고등어민박'은제주에서 가장...",2021-09-24,13,고등어민박,"['#데일리감성제주', '#데일리감성_제주—————————————————']",https://scontent-ssn1-1.cdninstagram.com/v/t51...,고등어민박,0
...,...,...,...,...,...,...,...,...
711,"#완주 #느린집 반려견동반이 가능한 완주 숙소, 느린집입니다.느린집은 완주의 어느 ...",2021-09-04,22,완주군,"['#완주', '#느린집', '#반려견동반펜션', '#파티룸', '#숙소하다_완주'...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,느린집,0
712,.유럽감성의 엔티쿠한 강릉_노벰버호텔!❣️동해바다뷰와 엔티크한 가구들로 꾸며진 내부...,2021-05-04,222,None,"['#강릉', '#노벰버', '#강릉숙소', '#노벰버호텔', '#강릉여행', '#...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,노벰버,0
713,"KBS2에서 방송됐던 ""한 번 다녀왔습니다""의 웨딩촬영지로 유명한 강릉_더브리앙뜨!...",2021-04-19,60,None,"['#강릉', '#더브리앙뜨', '#강릉숙소', '#강릉여행', '#강릉데이트', ...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,더브리앙뜨,0
714,.탁트인 창문뷰로 바라보는 포항바다!!🌊진짜 속이 뻥 뚫리는 느낌이햐~이불 속에서 ...,2021-03-29,102,None,"['#포항', '#테스226', '#포항숙소', '#포항여행', '#포항펜션', '...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,테스,0


In [14]:
def convert(x):
    return int(x.replace(',',''))

if tot_dataset.like.dtype == 'O':
    tot_dataset.like = tot_dataset.like.apply(lambda x : convert(x))
    
    
    
dg_cnt = Counter(tot_dataset.name)      # Counter 모듈로 name data count
li =[]
for name, cnt  in dg_cnt.items():            # 중복된 것만 뽑기 (cnt가 1이상이면 중복)
    if cnt > 1:
        li.append(name)
            

# 4. 행가져와서 like집계, content, tag list append
for i in li:           # 중복된 name 반복
    ex = tot_dataset.loc[tot_dataset.name == i]  # 반복된 name의 행을 가져온다
    ex.reset_index(drop=True,inplace=True)     # index reset
    
    content = ex.iloc[0,:].content
    date = ex.iloc[0,:].date
    like = ex.iloc[0,:].like
    place = ex.iloc[0,:].place
    tags = ex.iloc[0,:].tags
    name = ex.name[0]
    overlap = ex.iloc[0,:].overlap
    
    for i in range(len(ex)-1):
        content += ex.iloc[i+1,:].content
        like += ex.iloc[i+1,:].like
        tags += ex.iloc[i+1,:].tags
        overlap += ex.iloc[i+1,:].overlap
    
    
    # 5.차례대로 기존 행 지워내기
    indexNames = tot_dataset[tot_dataset['name'] == ex.name[0]].index
    tot_dataset.drop(indexNames, inplace=True) 
    tot_dataset = tot_dataset.append(pd.Series(data=[content,date,like,place,tags,name,overlap],index=tot_dataset.columns),ignore_index=True)

In [15]:
tot_dataset[tot_dataset.name == '온화한가']

,content,date,like,place,tags,imgUrl,name,overlap
496,#청도_온화한가경북 청도에 위치하고있는 조용하고 평화로운 독채풀빌라숙소 '온화한가'...,2021-08-08,4014,청도,"['#청도_온화한가경북', '#감성비앤비🏷청도로', '#감성비앤비_청도사진출처:',...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,온화한가,0


---

## Naver 장소 API

In [16]:
# naver application clientID,secret
client_id = "6E6mBjvxbdEgVk6Prqgd"
client_secret = "0ZjWnYiBjr"


# quote가 인코딩을 변경(UTF8로 url에 넘겨주기 위해서)
# 내가 입력한 값을 기준으로 parse
title_et = []
for name in tot_dataset.name:
    query = urllib.parse.quote(f'숙소+" "+{name}')
    idx = 0 # 요청 개수
    display = 10 # 100개 단위로 가져온다
    start = 1
    end = 100
    tmp=''

    url = "https://openapi.naver.com/v1/search/local?query=" + query \
        + "&display=" + str(display) \
        + "&start=" + str(start)


    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    # request 요청에 대한 결과를 response에 저장
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items']
        for item_index in range(0,len(items)):
            remove_tag = re.compile('<.*?>')
            title = re.sub(remove_tag, '', items[item_index]['title'])
            link = items[item_index]['link']
            caterory = items[item_index]['category']
            description = re.sub(remove_tag, '',items[item_index]['description'])
            telephone = items[item_index]['telephone']
            address = items[item_index]['address']
            roadAddress = items[item_index]['roadAddress']
            mapx = items[item_index]['mapx']
            mapy = items[item_index]['mapy']

            # 이름이 없는 경우
            if not title :
                title_et.append('')
            else: #'전통숙소' '호스텔''리조트''리조트부속건물'
                if caterory[:2] in ['숙박'] :  # 카테고리가 숙박인 경우
                    title = title.replace(' ','')
                    name = name.replace(' ','')
                    ratio = SequenceMatcher(None, title, name).ratio()
                    if ratio >= 0.7 : # title.replace(' ','') in name.replace(' ','')
                        tmp = address

        if tmp : title_et.append(tmp)
        else: title_et.append('')

    else:
        print("Error Code:" + rescode)

    time.sleep(0.1)


tot_dataset.drop('place',axis=1,inplace=True)
tot_dataset['place'] = title_et

In [17]:
tot_dataset.drop('imgUrl',axis=1,inplace=True)

In [18]:
tot_dataset['place'] = title_et

In [19]:
tot_dataset

,content,date,like,tags,name,overlap,place
0,"포항_선바우집호미곶 오션뷰 한옥 독채민박""선바우집""'선바우집'은가족단위로 머물기 좋...",2021-09-30,34,"['#데일리감성포항', '#데일리감성_포항경상북도로', '#데일리감성_경북—————...",선바우집,0,
1,"밀양_석주재프라이빗한 한옥 숙소""석주재""'석주재'는밀양의 프라이빗한 한옥 감성숙소로...",2021-09-27,22,"['#데일리감성밀양', '#데일리감성_밀양경상남도로', '#데일리감성_경남—————...",석주재,0,경상남도 밀양시 산외면 금곡리 45-5
2,"여수_컴온인동남아 이국적 분위기의 감성 숙소""컴온인""'컴온인'은동남아의 이국적인 분...",2021-09-26,21,"['#데일리감성여수', '#데일리감성_여수', '#데일리감성_전남——————————...",컴온인,0,
3,"강화_문스톤펜션강화도에서 가장 감성적인 숙소""문스톤펜션""'문스톤펜션'은강화도 동막해...",2021-09-26,14,"['#데일리감성강화', '#데일리감성_강화경기도로', '#데일리감성_경기——————...",문스톤펜션,0,제주특별자치도 제주시 구좌읍 월정리 235-1
4,"제주_고등어민박제주 동쪽 로맨틱한 감성 숙소""고등어민박""'고등어민박'은제주에서 가장...",2021-09-24,13,"['#데일리감성제주', '#데일리감성_제주—————————————————']",고등어민박,0,제주특별자치도 제주시 구좌읍 한동리 1338-8
...,...,...,...,...,...,...,...
711,"#완주 #느린집 반려견동반이 가능한 완주 숙소, 느린집입니다.느린집은 완주의 어느 ...",2021-09-04,22,"['#완주', '#느린집', '#반려견동반펜션', '#파티룸', '#숙소하다_완주'...",느린집,0,
712,.유럽감성의 엔티쿠한 강릉_노벰버호텔!❣️동해바다뷰와 엔티크한 가구들로 꾸며진 내부...,2021-05-04,222,"['#강릉', '#노벰버', '#강릉숙소', '#노벰버호텔', '#강릉여행', '#...",노벰버,0,충청남도 서산시 읍내동 102-17
713,"KBS2에서 방송됐던 ""한 번 다녀왔습니다""의 웨딩촬영지로 유명한 강릉_더브리앙뜨!...",2021-04-19,60,"['#강릉', '#더브리앙뜨', '#강릉숙소', '#강릉여행', '#강릉데이트', ...",더브리앙뜨,0,강원도 강릉시 사천면 노동리 75-1
714,.탁트인 창문뷰로 바라보는 포항바다!!🌊진짜 속이 뻥 뚫리는 느낌이햐~이불 속에서 ...,2021-03-29,102,"['#포항', '#테스226', '#포항숙소', '#포항여행', '#포항펜션', '...",테스,0,


In [20]:
tot_dataset.sort_values(by='like' ,ascending=False)

,content,date,like,tags,name,overlap,place
522,#거제_디스커버리풀빌라거제 위치한 전객실 오션뷰 숙소 '디스커버리풀빌라'객실에 들어...,2021-06-12,12781,"['#거제_디스커버리풀빌라거제', '#감성비앤비🏷거제로', '#감성비앤비_거제사진출...",디스커버리풀빌라,4,경상남도 거제시 장목면 시방리 782-1
595,"✅ #제주_과수원집소원재..서귀포시 한남리 조용한 시골 마을에 위치한 ""과수원집소원...",2021-04-07,11269,"['#제주_과수원집소원재..서귀포시', '#감성숙소_서귀포✔', '#감성숙소_제주....",과수원집소원재,0,제주특별자치도 서귀포시 남원읍 한남리 335-10
117,✅ #경주_천북빌리지드넓은 자연속에서 푸른 잔디가 펼쳐진 넓은 정원과 인피니티 수영...,2021-08-13,11073,"['#경주_천북빌리지드넓은', '#감성숙소_경주✔', '#감성숙소_경상북도📍천북빌리...",천북빌리지,0,
525,#남해_더파티펜션경남 남해에 위치하고있는 감성가득한 바베큐장이 매력적인 '더파티펜션...,2021-06-04,10092,"['#남해_더파티펜션경남', '#감성비앤비🏷남해로', '#감성비앤비_남해사진출처:'...",더파티펜션,0,충청남도 태안군 안면읍 창기리 1230-27
147,"✅ #제주_정취한가..작고 소박한 마을 안 오래된 제주 가옥을 리모델링해 만든 ""정...",2021-02-26,8936,"['#제주_정취한가..작고', '#감성숙소_애월읍✔', '#감성숙소_제주..', '...",정취한가,0,
...,...,...,...,...,...,...,...
133,✅ #가평_그레이어스..자연속에서 다른이들에게 방해받지 않고 우리만의 시간을 보낼 ...,2021-05-21,0,"['#가평_그레이어스..자연속에서', '#감성숙소_가평✔', '#감성숙소_경기도.....",그레이어스,0,경기도 가평군 가평읍 금대리 842-2
151,우리끼리 조용하고 안전하게 여행할수 있는 차박 명소 BEST 모음집!!오션뷰부터 은...,2021-02-05,0,"['#차박명소_감성숙소..📍태안', '#샛별해수욕장', '#장봉도', '#용머리공원...",태안,0,
164,"*현시점 그림의떡, 코로나끝나면 가야될곳 3탄*✅ #캐나다_메리어트폴스뷰호텔..미국...",2020-11-13,0,"['#캐나다_메리어트폴스뷰호텔..미국과', '#감성숙소_북아메리카✔', '#감성숙소...",메리어트폴스뷰호텔,0,
188,"*현시점 그림의떡, 코로나끝나면 가야될곳 2탄*✅ #이탈리아_알핀파노라마호텔후베르투...",2020-07-24,0,"['#이탈리아_알핀파노라마호텔후베르투스..아름다운', '#감성숙소_유럽✔', '#감...",알핀파노라마호텔후베르투스,0,


In [21]:
tot_dataset.to_csv('like_tot.csv')

In [22]:
tot_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 716 entries, 0 to 715
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  716 non-null    object
 1   date     716 non-null    object
 2   like     716 non-null    int64 
 3   tags     716 non-null    object
 4   name     716 non-null    object
 5   overlap  716 non-null    int64 
 6   place    716 non-null    object
dtypes: int64(2), object(5)
memory usage: 39.3+ KB
